# Trabalho Prático 2
**Grupo 22**

Alexis Correia - A102495 <br>
João Fonseca - A102512 <br>

## Enunciado
Considere a descrição da cifra A5/1 que consta no documento +Lógica Computacional: a Cifra A5/1 . Informação complementar pode ser obtida no artigo da Wikipedia. 

Pretende-se
1. Definir e codificar, em Z3 e usando o tipo BitVec para modelar a informação, uma FSM que descreva o gerador de chaves.
2. Considere as seguintes eventuais propriedades de erro:
    1. ocorrência de um “burst”  $\,\mathsf{0}^t\,$  ($t$ zeros) que ocorre em   $\,2^t\,$ passos ou menos.
    2. ocorrência de um “burst” de tamanho $\,t\,$ que repete um “burst” anterior no mesmo output em $2^{t/2}$  passos ou menos.
    Tente codificar estas propriedades e verificar se são acessíveis a partir de um estado inicial aleatoriamente gerado.

## Resolução

Formalmente uma FSM é um triplo $\,\Sigma\;\equiv\;\langle Q\,,\,I\,,\,\delta\rangle$  em que:
1. $Q$ é o conjunto (finito) de estados;
2. $I$ é o conjunto de estados iniciais, logo $I\in Q$;
3. $\delta$ é uma relação binária $Q\times Q$ designida **relação de transição**;

In [62]:
from pysmt.shortcuts import *
from pysmt.typing import BVType
import random

# Constantes
## Definindo o tamanho dos LFSRs
size0 = 19
size1 = 22
size2 = 23

## Posições de bits de controle de cada registrador
controlBit0 = 8
controlBit1 = 10
controlBit2 = 10

## Conatantes de transição
s0 = BV("1110010000000000000", size0)
s1 = BV("1100000000000000000000", size1)
s2 = BV("11100000000000010000000", size2)

def declare(i):
    s = {}
    s['lfsr0'] = Symbol('lfsr0_e'+str(i), BVType(size0))
    s['lfsr1'] = Symbol('lfsr1_e'+str(i), BVType(size1))
    s['lfsr2'] = Symbol('lfsr2_e'+str(i), BVType(size2))
    return s

def init(state): # Chave da cifra (aleatório)
    r0 = random.getrandbits(size0)
    A = Equals(state['lfsr0'],BV(r0, size0))

    r1 = random.getrandbits(size1)
    B = Equals(state['lfsr1'],BV(r1, size1))

    r2 = random.getrandbits(size2)
    C = Equals(state['lfsr2'],BV(r2, size2))
    return And(A,B,C)

def cBit(state):
    c0 = BVExtract(state['lfsr0'], controlBit0, controlBit0)
    c1 = BVExtract(state['lfsr1'], controlBit1, controlBit1)
    c2 = BVExtract(state['lfsr2'], controlBit2, controlBit2)
    if ((c0 & c1) | (c1 & c2) | (c0 & c2)): # BVAnd() e BVOr()
        r = BV(1,1)
    else:
        r = BV(0,1)
    return r

def trans(curr,prox):
    c = cBit(curr)
    t0 = And(Equals(BVExtract(curr['lfsr0'],controlBit0, controlBit0),c),
             Equals(prox['lfsr0'], BVXor(BVLShl(curr['lfsr0'],1),BVXor(curr['lfsr0'],s0))))
    t1 = And(Equals(BVExtract(curr['lfsr1'],controlBit1, controlBit1),c),
             Equals(prox['lfsr1'], BVXor(BVLShl(curr['lfsr1'],1),BVXor(curr['lfsr1'],s1))))
    t2 = And(Equals(BVExtract(curr['lfsr2'],controlBit2, controlBit2),c),
             Equals(prox['lfsr2'], BVXor(BVLShl(curr['lfsr2'],1),BVXor(curr['lfsr2'],s2))))
    
    return Or(And(t0,t1), And(t0,t2), And(t1,t2), And(t0,t1,t2)) #Or(t0, t1, t2)# 

def gera_traco(declare,init,trans): # k = 64
    states = [declare(i) for i in range(64)]
    solver = Solver(name = "z3")
    solver.add_assertion(init(states[0]))
    for i in range(63):
        solver.add_assertion(trans(states[i], states[i+1]))
    if solver.solve():
        for i,s in enumerate(states):
            r0 = format(solver.get_value(s['lfsr0']).constant_value(), f'0{size0}b')
            r1 = format(solver.get_value(s['lfsr0']).constant_value(), f'0{size1}b')
            r2 = format(solver.get_value(s['lfsr0']).constant_value(), f'0{size2}b')
            print(f"Estado {i}\n lfsr0:{r0} lfsr1:{r1} lfsr2:{r2}")
        pass
    else:
        print("> Not feasible.")
    #return states

gera_traco(declare, init, trans)

Estado 0
 lfsr0:1101011110100110011 lfsr1:0001101011110100110011 lfsr2:00001101011110100110011
Estado 1
 lfsr0:0000100010100111100 lfsr1:0000000100010100111100 lfsr2:00000000100010100111100
Estado 2
 lfsr0:1111110111101000100 lfsr1:0001111110111101000100 lfsr2:00001111110111101000100
Estado 3
 lfsr0:1110001000111001100 lfsr1:0001110001000111001100 lfsr2:00001110001000111001100
Estado 4
 lfsr0:1100001001001010100 lfsr1:0001100001001001010100 lfsr2:00001100001001001010100
Estado 5
 lfsr0:1001010111100000000 lfsr1:0001001010111100000000 lfsr2:00001001010111100000000
Estado 6
 lfsr0:0101101000100000000 lfsr1:0000101101000100000000 lfsr2:00000101101000100000000
Estado 7
 lfsr0:0000101001100000000 lfsr1:0000000101001100000000 lfsr2:00000000101001100000000
Estado 8
 lfsr0:1111101010100000000 lfsr1:0001111101010100000000 lfsr2:00001111101010100000000
Estado 9
 lfsr0:1110101111100000000 lfsr1:0001110101111100000000 lfsr2:00001110101111100000000
Estado 10
 lfsr0:1101100000100000000 lfsr1:0001101